In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [2]:
data = spark.read.csv("G:/titanic.csv ",inferSchema=True,header=True)

In [3]:
data.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [4]:
data.describe().show()

+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|summary|      PassengerId|           Survived|            Pclass|                Name|   Sex|               Age|             SibSp|              Parch|            Ticket|             Fare|Cabin|Embarked|
+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|  count|              891|                891|               891|                 891|   891|               714|               891|                891|               891|              891|  204|     889|
|   mean|            446.0| 0.3838383838383838| 2.308641975308642|                null|  null| 29.69911764705882|0.5230078563411896|0.38159371492704824|260318.54916792738| 32.20420

In [5]:
data.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [6]:
my_cols = data.select(['Survived',
 'Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Embarked'])

In [7]:
final_data = my_cols.na.drop()
final_data.count()

712

## Convert the Categorical data into Vector using OneHotEncoder and StringIndexer :

### => Convert Sex and Embarked columns using StringIndexer() and OneHotEncoder() respectively

In [8]:
from pyspark.ml.feature import VectorAssembler,VectorIndexer,OneHotEncoder,StringIndexer

In [9]:
sexIndexer = StringIndexer(inputCol="Sex", outputCol="Sex_index")
sexEncoder = OneHotEncoder(inputCol="Sex_index",outputCol="sexVector")

In [10]:
EmbIndexer = StringIndexer(inputCol="Embarked", outputCol="Embarked_index")
EmbEncoder = OneHotEncoder(inputCol="Embarked_index",outputCol="EmbarkedVector")

In [11]:
assembler = VectorAssembler(inputCols=['Pclass','sexVector','Age','SibSp','Parch','EmbarkedVector'],outputCol='feature')

## Pipeline

In [12]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline

In [13]:
RFC = RandomForestClassifier(featuresCol='feature',labelCol='Survived')

In [15]:
pipeline = Pipeline(stages=[sexIndexer,sexEncoder,EmbIndexer,EmbEncoder,assembler,RFC])

In [16]:
train, test = final_data.randomSplit([0.7,0.3])

In [17]:
fitting_model = pipeline.fit(train)

In [18]:
result = fitting_model.transform(test)

In [20]:
result.select("prediction", "Survived", "feature").show(5)


+----------+--------+--------------------+
|prediction|Survived|             feature|
+----------+--------+--------------------+
|       1.0|       0|[1.0,0.0,25.0,1.0...|
|       0.0|       0|[1.0,1.0,31.0,0.0...|
|       0.0|       0|[1.0,1.0,31.0,1.0...|
|       0.0|       0|[1.0,1.0,33.0,0.0...|
|       1.0|       0|[1.0,1.0,36.0,0.0...|
+----------+--------+--------------------+
only showing top 5 rows



## Multi-Class Classification Evaluator

In [21]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [25]:
my_eval = MulticlassClassificationEvaluator(labelCol='Survived',predictionCol='prediction',metricName='accuracy')

In [27]:
acc = my_eval.evaluate(result)

In [33]:
error = 1.0 - acc
error

0.18410041841004188

## Result :

### The Random Forest Classifier model seems to be performing decently with error of 0.18